# IPv4

<img src="res/ip01.png" width="500px">
<img src="res/ip02.png" width="500px">
<img src="res/ip03.png" width="500px">
<img src="res/ip04.png" width="500px">
<img src="res/ip05.png" width="500px">

1. IP fragment/defragment 本身有一些问题，比如乱序，id重复，丢失等等。这些问题ip层面是解决不了的，只能依赖上层协议（比如tcp）

<img src='res/ip06.png' width=500px>


-------------------------
-------------------------

## linux2.6/net/ipv4/ip_fragment.c

```c
/* Describe an entry in the "incomplete datagrams" queue. */
struct ipq {
	struct ipq	*next;		/* linked list pointers			*/
	struct list_head lru_list;	/* lru list member 			*/
	u32		user;
	u32		saddr;
	u32		daddr;
	u16		id;
	u8		protocol;
	u8		last_in;
#define COMPLETE		4
#define FIRST_IN		2
#define LAST_IN			1

	struct sk_buff	*fragments;	/* linked list of received fragments	*/
	int		len;		/* total length of original datagram	*/
	int		meat;
	spinlock_t	lock;
	atomic_t	refcnt;
	struct timer_list timer;	/* when will this queue expire?		*/
	struct ipq	**pprev;
	int		iif;
	struct timeval	stamp;
};
```

1. 这是一个双向链表的节点。有意思的点在于其 prev 的指针是一个指针的指针，存储的是prev节点中的next指针的地址。这个**实现很巧妙**

2. 每个queue的头需要存在hash table中。如果hash table直接存放节点struct，每次增删，rehash，代价很大。所以只能存放节点指针

3. 如果pprev不用 double 指针，那么每次操作比如删除，都要判断下当前是不是队列头，要对hash表中的位置进行单独操作。而用 double pointer，无论操作队列头还是中间节点，都是一样的

4. 这其实是kernel中hash link的统一的数据结构

### linux2.6/include/list.h

Hash List 结构

```c
/*
 * Double linked lists with a single pointer list head.
 * Mostly useful for hash tables where the two pointer list head is
 * too wasteful.
 * You lose the ability to access the tail in O(1).
 */

struct hlist_head {
	struct hlist_node *first;
};

struct hlist_node {
	struct hlist_node *next, **pprev;
};

```

普通 List 结构

```c
/*
 * Simple doubly linked list implementation.
 *
 * Some of the internal functions ("__xxx") are useful when
 * manipulating whole lists rather than single entries, as
 * sometimes we already know the next/prev entries and we can
 * generate better code by using them directly rather than
 * using the generic single-entry routines.
 */

struct list_head {
	struct list_head *next, *prev;
};

```



5. 具体可以参看 [kernel hlist](res/ip07.png)

	[kernel hlist详解](https://blog.csdn.net/hs794502825/article/details/24597773)


6. ipq是hash list的节点，为的是处理 hash 冲突，**而不是一个packet的不同段**。不同的ipq对应不同的 ip packet，每个ip packet（也就是每个节点）的fragments 在其内部的 fragments list存储，这是个sk_buff的list。

7. 当然，用double pointer带来的问题就是，没法直接访问上一个节点了。因为大部分情况并不需要访问上个节点的具体field，主要就是节点的增删，所以足够了。当然，如果一定要访问，可以用linux中经典的宏 container_of

* container_of 宏

```c
/**
 * container_of - cast a member of a structure out to the containing structure
 *
 * @ptr:	the pointer to the member.
 * @type:	the type of the container struct this is embedded in.
 * @member:	the name of the member within the struct.
 *
 */
#define container_of(ptr, type, member) ({			\
        const typeof( ((type *)0)->member ) *__mptr = (ptr);	\
        (type *)( (char *)__mptr - offsetof(type,member) );})
```

* offsetof 宏

```c
#ifdef __compiler_offsetof
#define offsetof(TYPE,MEMBER) __compiler_offsetof(TYPE,MEMBER)
#else
#define offsetof(TYPE, MEMBER) ((size_t) &((TYPE *)0)->MEMBER)
#endif
```

-----------------